In [1]:
import datetime
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score, train_test_split


df = pd.read_csv('features.csv', index_col='match_id')
df_test = pd.read_csv('features_test.csv', index_col='match_id')
df_copy = deepcopy(df) 

In [2]:
col = ['duration', 'radiant_win', 'tower_status_radiant', "tower_status_dire",
       'barracks_status_radiant', 'barracks_status_dire']

# удалил признаки связанные с итогами матча
df.drop(col, axis=1, inplace=True, errors='ignore')

In [3]:
# посчитал количество пропусков
c = df.count()

j = 0
names_count = np.empty(12)
for i in range(len(c)):
    if c[i] < 97230:
        names_count[j] = i # записал название признаков имеющих пропуски 
        j += 1
        print('Number of row:', c.index[i], 'count:', c[i])

Number of row: first_blood_time count: 77677
Number of row: first_blood_team count: 77677
Number of row: first_blood_player1 count: 77677
Number of row: first_blood_player2 count: 53243
Number of row: radiant_bottle_time count: 81539
Number of row: radiant_courier_time count: 96538
Number of row: radiant_flying_courier_time count: 69751
Number of row: radiant_first_ward_time count: 95394
Number of row: dire_bottle_time count: 81087
Number of row: dire_courier_time count: 96554
Number of row: dire_flying_courier_time count: 71132
Number of row: dire_first_ward_time count: 95404


In [4]:
mean_df = df.mean(axis=0)

# df.fillna(0.00000000001, inplace=True) # пробовал заменять пропуски
# на очень маленькое значение


# заменяю пропуски на среднее значение по этому столбцу

for i in range(len(names_count)):
    # print(c.index[names_count[i]], c[int(names_count[i])])
    df[c.index[names_count[i]]].fillna(mean_df[int(names_count[i])], inplace=True)
    # print(df[c.index[names_count[i]]])
    # print("---------------------------------------------------------------")

# df.fillna(0, axis=0, inplace=True) # пробовал заменять пропуски на нули    


# уменьшал обучающую выборку, засчет этого нашел ответ на вопрос
# о том как ускорить время обучения градиентного бустинга

# x_train, x_test, y_train, y_test =  train_test_split(df, df_copy['radiant_win'], test_size=0.6, random_state=241)


In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)

# использовал закоментированный код ниже чтобы проверить качество и время обучения
# для разного кол-ва деревьев

# для обучения использовал также выше указанную уменьшенную обучающую выборку с
# разнымми значениями
# и пробовал уменьшить max_depth - время обучения реально уменьшается

# n_est = 10
# for i in range(5):
#     clf = GradientBoostingClassifier(n_estimators=n_est, verbose=True,
#                                      random_state=241, max_depth=2)
#     # clf.fit(x_train, y_train)
#
#     start_time = datetime.datetime.now()
#
#     # gbc = np.mean(cross_val_score(clf, x_train, np.ravel(y_train, order = 'C'), cv=kf, scoring='roc_auc'))
#     gbc = np.mean(cross_val_score(clf, df, df_copy['radiant_win'],
#                                   cv=kf, scoring='roc_auc'))
#
#     n_est = n_est + 10
#     print('Time elapsed:', datetime.datetime.now() - start_time)
#     print(gbc, n_est)

clf = GradientBoostingClassifier(n_estimators=30, verbose=True, random_state=241)

start_time = datetime.datetime.now()

gbc = np.mean(cross_val_score(clf, df, df_copy['radiant_win'] ,
                              cv=kf, scoring='roc_auc'))

print('Time elapsed:', datetime.datetime.now() - start_time)
print(gbc)

      Iter       Train Loss   Remaining Time 
         1           1.3786           17.44s
         2           1.3730           16.80s
         3           1.3680           16.18s
         4           1.3634           15.60s
         5           1.3592           15.00s
         6           1.3547           14.39s
         7           1.3503           13.80s
         8           1.3464           13.21s
         9           1.3425           12.62s
        10           1.3385           12.02s
        20           1.3091            6.07s
        30           1.2891            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3781           18.37s
         2           1.3725           17.69s
         3           1.3674           16.92s
         4           1.3628           16.56s
         5           1.3585           15.89s
         6           1.3539           15.19s
         7           1.3498           14.52s
         8           1.3454           13.83s
        

In [6]:
names = df.columns
scal = preprocessing.StandardScaler()
df_scaled = scal.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=names)

reg = LogisticRegression(penalty='l2', C=0.01, random_state=241)

start_time = datetime.datetime.now()
lg = np.mean(cross_val_score(reg, df_scaled, df_copy['radiant_win'],
                             cv=kf, scoring='roc_auc'))
print('Time elapsed:', datetime.datetime.now() - start_time)

print (lg)

Time elapsed: 0:00:01.940498
0.7167263303131431


In [7]:
# убираю категориальные признаки 
coll = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

df_scaled.drop(coll, axis=1, inplace=True, errors='ignore')

start_time = datetime.datetime.now()
lg_1 = np.mean(cross_val_score(reg, df_scaled, df_copy['radiant_win'],
                               cv=kf, scoring='roc_auc'))
print('Time elapsed:', datetime.datetime.now() - start_time)

print (lg_1)

Time elapsed: 0:00:02.226392
0.716783013898749


In [11]:
# ниже код собирает мешок слов после подсчета количества уникальных героев 
# и добавляем новые признаки в обучающую выборку

hero_count = pd.unique(df[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                           'd1_hero', 'd3_hero', 'd4_hero', 'd5_hero']].values.ravel())

x_pick = np.zeros((df.shape[0], 112))

for i, match_id in enumerate(df.index):
    for p in range(5):
        x_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        x_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

x_pick = pd.DataFrame(x_pick)
data = np.concatenate([df_scaled, x_pick], axis=1)

In [12]:
start_time = datetime.datetime.now()
lg_2 = np.mean(cross_val_score(reg, data, df_copy['radiant_win'], cv=kf, scoring='roc_auc'))
print('Time elapsed:', datetime.datetime.now() - start_time)

print (lg_2)

Time elapsed: 0:00:06.154195
0.7518403366123803


In [13]:
# далее проделывается все те же преобразования c обучающей выборкой
# только для df_test
c_tst = df_test.count()

j = 0
names_tst_count = np.empty(12)
for i in range(len(c_tst)):
    if c_tst[i] < 17177:
        names_tst_count [j] = i
        j += 1
        # print('Number of row:', c_tst.index[i], 'count:', c_tst[i])

mean_tst_df = df_test.mean(axis=0) 

In [14]:
for i in range(len(names_tst_count)):
    df_test[c_tst.index[names_tst_count[i]]].fillna(mean_tst_df[int(names_tst_count[i])], inplace=True)

df_tst_scaled = scal.fit_transform(df_test)
df_tst_scaled = pd.DataFrame(df_tst_scaled, columns=names)

df_tst_scaled.drop(coll, axis=1, inplace=True, errors='ignore')

In [15]:
x_pick_tst = np.zeros((df_test.shape[0], 112))

for i, match_id in enumerate(df_test.index):
    for p in range(5):
        x_pick_tst[i, df_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        x_pick_tst[i, df_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

x_pick_tst = pd.DataFrame(x_pick_tst)
data_tst = np.concatenate([df_tst_scaled, x_pick_tst], axis=1)

In [16]:
# обучаем логистическую регрессию на преобразованных данных
reg.fit(data, df_copy['radiant_win'])
final = reg.predict_proba(data_tst)[:, 1]

print (final.max())
print (final.min())





0.9959619261912386
0.008616422488704298
